### Ensuring Consistency Across Training & Inference Datasets: Feature Scaling
**Question**: Load a dataset (e.g., Boston Housing) and perform feature scaling. Ensure the
same scaling is applied during model inference with new data.

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def load_boston_data():
    try:
        boston = load_boston()
        X = pd.DataFrame(boston.data, columns=boston.feature_names)
        y = pd.Series(boston.target, name='MEDV')

        # Validate no missing values and correct dtypes
        if X.isnull().any().any():
            raise ValueError("Feature data contains missing values.")
        if y.isnull().any():
            raise ValueError("Target contains missing values.")
        if not np.all([np.issubdtype(dtype, np.number) for dtype in X.dtypes]):
            raise TypeError("Feature data contains non-numeric values.")

        return X, y
    except Exception as e:
        print(f"Error loading Boston dataset: {e}")
        return None, None

def scale_features(X_train, X_infer):
    if X_train is None or X_infer is None:
        raise ValueError("Input data cannot be None.")

    # Check for NaNs and valid shapes
    if X_train.isnull().any().any() or X_infer.isnull().any().any():
        raise ValueError("Input contains missing values.")
    if X_train.shape[1] != X_infer.shape[1]:
        raise ValueError("Train and inference feature dimensions do not match.")

    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_infer_scaled = pd.DataFrame(scaler.transform(X_infer), columns=X_infer.columns, index=X_infer.index)

    return X_train_scaled, X_infer_scaled, scaler

def test_scaling_consistency():
    print("Running scaling consistency tests...")

    X, y = load_boston_data()
    if X is None or y is None:
        print("Skipping tests due to data loading failure.")
        return

    X_train, X_infer, y_train, y_infer = train_test_split(X, y, test_size=0.2, random_state=42)

    # Introduce a NaN in inference to test error handling (comment out for normal run)
    # X_infer.iloc[0, 0] = np.nan

    try:
        X_train_scaled, X_infer_scaled, scaler = scale_features(X_train, X_infer)
    except Exception as e:
        print(f"Scaling error caught as expected: {e}")
        return

    # Test means ~ 0 and std ~ 1 for training data
    means = X_train_scaled.mean()
    stds = X_train_scaled.std()
    assert all(np.abs(means) < 1e-6), f"Training features mean not ~0: {means}"
    assert all(np.abs(stds - 1) < 1e-6), f"Training features std not ~1: {stds}"

    # Test inference scaled data uses same scaler (no refit)
    # Means and stds of inference data likely different, but scaled with same params
    infer_means = X_infer_scaled.mean()
    infer_stds = X_infer_scaled.std()
    print("Inference scaled data means (approx):")
    print(infer_means.head())
    print("Inference scaled data std dev (approx):")
    print(infer_stds.head())

    # Boundary condition: empty dataframe input
    try:
        scale_features(pd.DataFrame(), pd.DataFrame())
    except ValueError as e:
        print(f"Empty dataframe test caught: {e}")

    print("Scaling consistency tests passed.\n")

if __name__ == "__main__":
    # Load data
    X, y = load_boston_data()
    if X is None or y is None:
        print("Failed to load data.")
        exit(1)

    # Split train/inference
    X_train, X_infer, y_train, y_infer = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale
    try:
        X_train_scaled, X_infer_scaled, scaler = scale_features(X_train, X_infer)
    except Exception as e:
        print(f"Error during scaling: {e}")
        exit(1)

    print("Sample scaled training data:")
    print(X_train_scaled.head())

    print("\nSample scaled inference data:")
    print(X_infer_scaled.head())

    # Run tests
    test_scaling_consistency()

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [2]:
# write your code from here  
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def load_data():
    try:
        # Load Boston Housing data
        data = load_boston()
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.Series(data.target, name='MEDV')
        if X.empty or y.empty:
            raise ValueError("Dataset is empty.")
        return X, y
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None, None

def scale_features(X_train, X_infer):
    if X_train is None or X_infer is None:
        print("Invalid data for scaling.")
        return None, None, None

    if not isinstance(X_train, pd.DataFrame) or not isinstance(X_infer, pd.DataFrame):
        raise TypeError("Inputs must be pandas DataFrames.")

    scaler = StandardScaler()
    try:
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        X_infer_scaled = pd.DataFrame(scaler.transform(X_infer), columns=X_infer.columns)
    except Exception as e:
        print(f"Error during scaling: {e}")
        return None, None, None

    return X_train_scaled, X_infer_scaled, scaler

def test_scaling_consistency():
    print("\nRunning scaling consistency tests...")

    X, y = load_data()
    if X is None:
        print("Skipping tests due to data loading failure.")
        return

    # Split data into train and inference sets
    X_train, X_infer, y_train, y_infer = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train_scaled, X_infer_scaled, scaler = scale_features(X_train, X_infer)

    # Check none is None
    assert X_train_scaled is not None, "Training data scaling failed"
    assert X_infer_scaled is not None, "Inference data scaling failed"
    assert scaler is not None, "Scaler not returned"

    # Mean and std of scaled training data should be ~0 and 1
    means = X_train_scaled.mean().round(6)
    stds = X_train_scaled.std(ddof=0).round(6)  # population std

    print("Training scaled feature means (should be ~0):")
    print(means)
    print("Training scaled feature std devs (should be ~1):")
    print(stds)

    # Test inference scaled data was scaled using same scaler
    # For each feature, check inference mean is roughly near zero (depends on sample)
    infer_means = X_infer_scaled.mean()
    print("Inference scaled feature means (may deviate from 0):")
    print(infer_means)

    print("Scaling consistency tests passed.\n")

if __name__ == "__main__":
    X, y = load_data()
    if X is not None:
        # Split into train and inference sets
        from sklearn.model_selection import train_test_split
        X_train, X_infer, y_train, y_infer = train_test_split(X, y, test_size=0.2, random_state=42)

        # Scale features
        X_train_scaled, X_infer_scaled, scaler = scale_features(X_train, X_infer)

        print("Sample scaled training data:")
        print(X_train_scaled.head())

        print("\nSample scaled inference data:")
        print(X_infer_scaled.head())

        # Run tests
        test_scaling_consistency()

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: